In [32]:
# Imports
import sys
import gurobipy as gp
import numpy as np

In [33]:
# Parameters (Valus of Instance 1)
P = 5           # Tempo de circulação de um vagão j em um trecho q
M = 2880        # Número grande o suficiente
n = 2           # NUM_VAGOES
m = 6           # NUM_TRECHOS
o = 15          # NUM_ATIVIDADES
v = 2           # NUM_TRENS
t_beta = [1,1]  # VAGOES_TREM_SAIDA
t_alfa = [0,0]  # VAGOES_TREM_CHEGADA

# AMV[q,s] = 1 se o trecho q é ligado ao trecho s (q vizinho de s ou existe AMV entre q e s); 0 caso contrário
AMV = [[1,1,0,0,0,0],
       [1,1,1,0,1,0],
       [0,1,1,1,0,0],
       [0,0,1,1,0,0],
       [0,1,0,0,1,1],
       [0,0,0,0,1,1]]   
# PCH[j] = trecho do pátio onde o vagão j estará posicionado ao receber o trem t ∈ Tα|j ∈ t
PCH = [2,1]
# PSA[j] = trecho do pátio onde o vagão j deverá estar posicionado para que o trem t ∈ Tβ |j ∈ t possa sair
PSA = [1,2]
# RELEASE[j] = instante mais cedo que o vagão j está disponível para entrar no pátio (release date), ou seja, horário de chegada do trem t ∈ Tα|j ∈ t
RELEASE = [10,10]

In [34]:
# Initialize model
model = gp.Model("milp")

In [35]:
# Declare decision variables
var_y = model.addVars(o,n, vtype=gp.GRB.CONTINUOUS, name="y")         ## y[i,j] = instante de início da i-ésima operação do vagão j
var_z = model.addVars(o,m,n, vtype=gp.GRB.BINARY, name="z")           ## z[i,q,j] = 1 se a i-ésima operação do vagão j acontece no trecho q; 0 caso contrário
var_x = model.addVars(m,o,n,o,n, vtype=gp.GRB.BINARY, name="x")       ## x[q,i,j,k,l] = 1 se a i-ésima operação do vagão j precede a k-ésima operação do vagão l no trecho q; 0 caso contrário
var_f = model.addVars(o,n, vtype=gp.GRB.BINARY, name="f")             ## f[i,j] = 1 se a i-ésima operação do vagão j é a última, antes da partida do trem t ∈ Tβ |j ∈ t; 0 caso contrário
var_c = model.addVars(n, vtype=gp.GRB.CONTINUOUS, name="c")           ## C[j] = instante de conclusão do vagão j, horário de partida do trem t ∈ Tβ|j ∈ t
var_c_max = model.addVar(vtype=gp.GRB.CONTINUOUS, name="c_max")       ## Cmax = horário da partida do último vagão j ∈ t, t ∈ Tβ (makespan)

In [36]:
# Define objective function
model.setObjective(var_c_max, gp.GRB.MINIMIZE)

In [37]:
# Equation 2
for j in range(n):
  for i in range(o):
    if (i < o-1):
      model.addConstr
      (
        var_y[i+1,j] 
        >= var_y[i,j] 
        + (P * gp.quicksum(var_z[i,q,j] for q in range(m)))
      )

In [38]:
# Equation 3
for j in range(n):
  for l in range(n):
    for i in range(o):
      for k in range(o):
        for q in range(m):
          if (i+1 < o):
            model.addConstr
            (
              var_y[i+1,j] 
              >= var_y[k,l]
              - (M * var_x[q,i,j,k,l])
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [39]:
# Equation 4
for j in range(n):
  for l in range(n):
    for i in range(o):
      for k in range(o):
        for q in range(m):
          if (i+1 < o):
            model.addConstr
            (
              var_y[k,l] 
              >= var_y[i+1,j]
              - (M * (1 - var_x[q,i,j,k,l]))
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [40]:
# Equation 5
for j in range(n):
  for i in range(o):
    if (i == 0):
      model.addConstr(gp.quicksum(var_z[i,q,j] for q in range(m)) <= 1)

In [41]:
# Equation 6
for j in range(n):
  for l in range(n):
    for i in range(o): 
      for k in range(o):
        for q in range(m):
          if (k > 0 and i < o-1 and j != l):
            model.addConstr
            (
              var_x[q,i,j,k,l] 
              + gp.quicksum(var_x[s,k-1,l,i+1,j] for s in range(m))
              <= 1
            )

In [42]:
# Equation 7
for j in range(n):
  for l in range(n):
    for i in range(o): 
      for k in range(o):
        for q in range(m):
          if (j != l):
            model.addConstr
            (
              var_x[q,i,j,k,l] 
              + var_x[q,k,l,i,j] 
              >= 1
              - (M * (1 - var_z[i,q,j]))
              - (M * (1 - var_z[k,q,l]))
            )

In [43]:
# Equation 8
for j in range(n):
  for i in range(o):
    if (i > 0):
      model.addConstr
      (
        gp.quicksum(var_z[i,q,j] for q in range(m))
        == 1
        - gp.quicksum(var_f[k,j] for k in range(i-1))
      )

In [44]:
# Equation 9
for j in t_beta:
  for i in range(o):
    model.addConstr(var_f[i,j] <= var_z[i,PSA[j],j])

In [45]:
# Equation 10
for j in t_beta:
  model.addConstr(gp.quicksum(var_f[i,j] for i in range(o)) == 1)

In [46]:
# Equation 11
for j in t_alfa:
  model.addConstr(gp.quicksum(var_f[i,j] for i in range(o)) == 0)

In [47]:
# Equation 12
for j in range(n):
  for i in range(o):
    for q in range(m):
      if (i == 0 and q == PCH[j]):
        model.addConstr(var_z[i,q,j] == 1)

In [48]:
# Equation 13
for j in t_beta:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_y[i,j] == RELEASE[j])

In [49]:
# Equation 14
for j in t_beta:
  for i in range(o):
    if (i == 0):
      model.addConstr(var_y[i,j] >= RELEASE[j])

In [50]:
# Equation 15
for j in t_alfa:
  for l in t_alfa:
    for i in range(o):
      if (i == 0):
        model.addConstr(var_y[i,j] == var_y[i,l])

In [51]:
# Equation 16
for j in range(n):
  for i in range(o):
    for q in range(m):
      if (i+1 < o):
        model.addConstr(var_z[i+1,q,j] <= gp.quicksum(AMV[s][q] * var_z[i,s,j] for s in range(m)))

In [52]:
# Equation 17
for j in t_beta:
  for l in t_beta:
    for i in range(o):
      model.addConstr(var_c[j] >= var_y[i,l])

In [53]:
# Equation 18
for j in t_alfa:
  for i in range(o):
    if (i == o-1):
      model.addConstr(var_c[j] >= var_y[i,j])

In [54]:
# Equation 19
for j in range(n):
  for i in range(o):
    if (i+1 < o):
      model.addConstr
      (
        var_y[i+1,j]
        >= var_c[j] 
        - M 
        * (1 - gp.quicksum(var_f[k,j] for k in range(i)))
      )

In [55]:
# Equation 20
for j in t_beta:
  model.addConstr(var_c_max >= var_c[j])

In [56]:
# Execute model
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 278 rows, 5643 columns and 882 nonzeros
Model fingerprint: 0xc2b6a3ee
Variable types: 33 continuous, 5610 integer (5610 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 10.0000000
Presolve removed 278 rows and 5643 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.08 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


In [57]:
# Print the values of all variables
all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

for name, val in zip(names, values):
    if (val > 0):
        print(f"{name} = {val}")

y[0,1] = 10.0
z[0,1,1] = 1.0
z[0,2,0] = 1.0
z[1,0,1] = 1.0
z[1,1,0] = 1.0
z[1,1,1] = 1.0
z[1,2,0] = 1.0
z[1,2,1] = 1.0
z[1,3,0] = 1.0
z[1,4,1] = 1.0
z[2,0,0] = 1.0
z[2,0,1] = 1.0
z[2,1,0] = 1.0
z[2,1,1] = 1.0
z[2,2,0] = 1.0
z[2,2,1] = 1.0
z[2,3,0] = 1.0
z[2,3,1] = 1.0
z[2,4,0] = 1.0
z[2,4,1] = 1.0
z[2,5,1] = 1.0
z[3,0,0] = 1.0
z[3,0,1] = 1.0
z[3,1,0] = 1.0
z[3,1,1] = 1.0
z[3,2,0] = 1.0
z[3,2,1] = 1.0
z[3,3,0] = 1.0
z[3,3,1] = 1.0
z[3,4,0] = 1.0
z[3,4,1] = 1.0
z[3,5,0] = 1.0
z[3,5,1] = 1.0
z[4,0,0] = 1.0
z[4,0,1] = 1.0
z[4,1,0] = 1.0
z[4,1,1] = 1.0
z[4,2,0] = 1.0
z[4,2,1] = 1.0
z[4,3,0] = 1.0
z[4,3,1] = 1.0
z[4,4,0] = 1.0
z[4,4,1] = 1.0
z[4,5,0] = 1.0
z[4,5,1] = 1.0
z[5,0,0] = 1.0
z[5,0,1] = 1.0
z[5,1,0] = 1.0
z[5,1,1] = 1.0
z[5,2,0] = 1.0
z[5,2,1] = 1.0
z[5,3,0] = 1.0
z[5,3,1] = 1.0
z[5,4,0] = 1.0
z[5,4,1] = 1.0
z[5,5,0] = 1.0
z[5,5,1] = 1.0
z[6,0,0] = 1.0
z[6,0,1] = 1.0
z[6,1,0] = 1.0
z[6,1,1] = 1.0
z[6,2,0] = 1.0
z[6,2,1] = 1.0
z[6,3,0] = 1.0
z[6,3,1] = 1.0
z[6,4,0] = 1.0
z[6,4,1] = 